# Andy's Final Assignment of Segmenting and Clustering

Adding two new columns to the dataframe 

In [1]:

!pip install BeautifulSoup4
!pip install requests

We needed install the previous libraries.

In [2]:

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np


source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')


data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    

    if (index == 0):
        columns = section
    else:
        data.append(section)


df = pd.DataFrame(data = data,columns = columns)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


In [3]:
#Remove 'Not assigned' in column  Borough
df = df[df['Borough'] != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [4]:
# postal code area:
df["Neighbourhood"] = df.groupby("Postcode")\
["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))

#remove duplicates:
df =df.drop_duplicates()

df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
7,M7A,Downtown Toronto,Queen's Park


In [5]:
# If a cell has a borough but a Not assigned neighborhood..:
df['Neighbourhood'].replace("Not assigned", df["Borough"],inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
7,M7A,Downtown Toronto,Queen's Park


In [12]:
df.shape

(103, 3)

In [6]:
import pandas as pd
data=pd.read_csv("Geospatial_Coordinates.csv")
data.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
df['Latitude']=data['Latitude']
df['Longitude']=data['Longitude']
df.head(100)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.763573,-79.188711
3,M4A,North York,Victoria Village,43.770992,-79.216917
4,M5A,Downtown Toronto,Harbourfront,43.773136,-79.239476
5,M6A,North York,"Lawrence Heights, Lawrence Manor",43.744734,-79.239476
7,M7A,Downtown Toronto,Queen's Park,43.711112,-79.284577
...,...,...,...,...,...
245,M1X,Scarborough,Upper Rouge,NaN,NaN
248,M4X,Downtown Toronto,"Cabbagetown, St. James Town",NaN,NaN
250,M5X,Downtown Toronto,"First Canadian Place, Underground city",NaN,NaN
254,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",NaN,NaN


In [1]:
df['Latitude']


NameError: name 'df' is not defined

In [21]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

The geograpical coordinate of New York City are 43.653963, -79.387207.


In [25]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [28]:

map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'],\
        df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

ValueError: Location values cannot contain NaNs, got:
[nan, nan]